# Runtime
Calls the ML models, handles training, loads in the dataset, using all the other methods present in ./Experiments and ./Models

## Methods and Libraries

In [42]:
import os
import torch
import wandb

# DataLoader, Hyperparameters, Train/Test
%run -i defungi.ipynb
%run -i train.ipynb
%run -i config.ipynb

# Model Architecture
%run -i ../Models/net_conv.ipynb
%run -i ../Models/net_conv_rf.ipynb
%run -i ../Models/net_rnn.ipynb
%run -i ../Models/net_xformer.ipynb

## Argument and Configuration Handler 

In [31]:
# Determine MODEL to use
MODEL = 'null'
models = ['CDIL', 'DIL', 'TCN', 'CNN', 'Deformable', 'LSTM', 'GRU']
while MODEL not in models:
    print('Select one of the following models to train on the dataset: ', end='')
    for i in models:
        print(i, ' ', end='')
    MODEL = input('Selection:')


# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Config parameters
cfg_training = config['training']
cfg_model = config['models']

BATCH_SIZE = cfg_training['batch_size']
CLASS = cfg_model['n_class']
SEQ_LEN = cfg_model['n_length']
FIX_LENGTH = cfg_model['fix_length']
USE_EMBED = cfg_model['use_embedding']
CHAR_COCAB = cfg_model['vocab_size']
INPUT_SIZE = cfg_model['dim']

LAYER = cfg_model['cnn_layer']
NHID = cfg_model['cnn_hidden']
KERNEL_SIZE = cfg_model['cnn_ks']

Select one of the following models to train on the dataset: CDIL  DIL  TCN  CNN  Deformable  LSTM  GRU  

Selection: CDIL


## Load WANDB for evaluation metrics & visualization

In [5]:
# Using WANDB for visualization of validation and performance
wandb.init(project='DeFungi', name=MODEL, entity="jacob-davidian")
WANDB = wandb

wandb: Currently logged in as: hawkhobo (jacob-davidian). Use `wandb login --relogin` to force relogin


## Load and Prepare Dataset

In [43]:
# Select between Grayscale or RGB dataloading
while True:
    rgb = input("Enter 3 for RGB images, 1 for grayscale: ")
    if rgb in {'1', '3'}:
        color_option = (rgb == '3')
        break
    else:
        print("Invalid input. Please enter 1 or 3.")
    
train_loader, test_loader = DeFungiDataset(
                                  '../EDA/Dataset', 
                                 use_rgb=color_option,
                             batch_size=BATCH_SIZE, 
                                     test_size=0.2)

print(f'train_loader contents: {train_loader}\n test_loader contents: {test_loader}')

Enter 3 for RGB images, 1 for grayscale:  3


Batch 1:
Inputs Type: torch.LongTensor
Labels Type: torch.LongTensor
Inputs Shape: torch.Size([32, 750000])
Labels Shape: torch.Size([32])
train_loader contents: <torch.utils.data.dataloader.DataLoader object at 0x7ff8e8184190>
 test_loader contents: <torch.utils.data.dataloader.DataLoader object at 0x7ff8e8186890>


# Model Selection

In [41]:
# Model selection with configs 
if MODEL == 'CDIL' or MODEL == 'DIL' or MODEL == 'TCN' or MODEL == 'CNN':
    LAYER = cfg_model['cnn_layer']
    NHID = cfg_model['cnn_hidden']
    KERNEL_SIZE = cfg_model['cnn_ks']
    net = CONV(MODEL, INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, False, False, USE_EMBED, CHAR_COCAB, FIX_LENGTH)
    receptive_field(seq_length=SEQ_LEN, model=MODEL, kernel_size=KERNEL_SIZE, layer=LAYER)
elif MODEL == 'Deformable':
    LAYER = cfg_model['cnn_layer']
    NHID = cfg_model['cnn_hidden']
    KERNEL_SIZE = cfg_model['cnn_ks']
    net = CONV('CNN', INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, True, False, USE_EMBED, CHAR_COCAB, FIX_LENGTH)
    receptive_field(seq_length=SEQ_LEN, model=MODEL, kernel_size=KERNEL_SIZE, layer=LAYER)
elif MODEL == 'LSTM' or MODEL == 'GRU':
    LAYER = cfg_model['rnn_layer']
    NHID = cfg_model['rnn_hidden']
    net = RNN(MODEL, INPUT_SIZE, CLASS, NHID, LAYER, USE_EMBED, CHAR_COCAB, FIX_LENGTH)
    
net = CONV(MODEL, INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, False, False, USE_EMBED, INPUT_SIZE, FIX_LENGTH)

# Create sample inputs
sample_input = torch.randint(0, INPUT_SIZE, (BATCH_SIZE, SEQ_LEN))

outputs = net(sample_input)

net = net.to(device)
para_num = sum(p.numel() for p in net.parameters() if p.requires_grad)

print(f'architecture of {MODEL}: \n{net}')

position: 512
layer:1 	 distance:3 	 size:3
layer:2 	 distance:7 	 size:7
layer:3 	 distance:15 	 size:15
layer:4 	 distance:31 	 size:31
layer:5 	 distance:63 	 size:63
layer:6 	 distance:127 	 size:127
layer:7 	 distance:255 	 size:255
layer:8 	 distance:511 	 size:511
layer:9 	 distance:1023 	 size:1023

Input Shape: torch.Size([32, 1024])
After Embedding Shape: torch.Size([32, 1024, 64])
After Permute Shape: torch.Size([32, 64, 1024])
architecture of CDIL: 
CONV(
  (embedding): Embedding(64, 64)
  (conv): ConvPart(
    (conv_net): Sequential(
      (0): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (1): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (2): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,), padding_mode=circ

## Perform Training

In [29]:
TrainModel(
    model=net,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=cfg_training['epoch'],
    learning_rate=cfg_training['lr'],
    wandb=WANDB
)


Epoch 1/100:   0%|                                    | 0/11 [00:04<?, ?batch/s]


IndexError: index out of range in self

## Pass to Front-End

In [ ]:
# Save Training Results for front-end
filename = 'fungi_classifier.sav'
pickle.dump(f'DeFungi_{MODEL}', open(filename, 'wb'))